In [19]:
from langchain_neo4j import GraphCypherQAChain, Neo4jGraph
from langchain_groq import ChatGroq

graph = Neo4jGraph(url = "neo4j+s://d1a48238.databases.neo4j.io", username="neo4j", password = "QdSbKKtvoMMJ2r7QqQzxnwTenh8Ps7paoa3_qriKKqU")

In [20]:
graph.query(
    """
MERGE (m:Movie {name:"Top Gun", runtime: 120})
WITH m
UNWIND ["Tom Cruise", "Val Kilmer", "Anthony Edwards", "Meg Ryan"] AS actor
MERGE (a:Actor {name:actor})
MERGE (a)-[:ACTED_IN]->(m)
"""

)

[]

In [21]:
graph.refresh_schema()

In [22]:
print(graph.schema)

Node properties:
Document {doc_id: STRING, title: STRING, content: STRING}
Person {person_id: STRING, name: STRING, bio: STRING}
Movie {name: STRING, runtime: INTEGER}
Actor {name: STRING}
Relationship properties:

The relationships:
(:Person)-[:AUTHORED]->(:Document)
(:Actor)-[:ACTED_IN]->(:Movie)


In [23]:
enchanced_graph = Neo4jGraph(
    url = "neo4j+s://d1a48238.databases.neo4j.io",
    username = "neo4j",
    password = "QdSbKKtvoMMJ2r7QqQzxnwTenh8Ps7paoa3_qriKKqU",
    enhanced_schema=True,
)

print(enchanced_graph.schema)

Node properties:
- **Document**
  - `doc_id`: STRING Available options: ['doc1', 'doc2', 'doc3']
  - `title`: STRING Available options: ['GraphRAG overview', 'Neo4j basics', 'Hybrid search patterns']
  - `content`: STRING Available options: ['GraphRAG combines graph knowledge with retrieval a', 'Neo4j is a graph database where nodes are connecte', 'Combine BM25/fulltext and vector similarity to get']
- **Person**
  - `person_id`: STRING Available options: ['alice']
  - `name`: STRING Available options: ['Alice']
  - `bio`: STRING Available options: ['Alice is a knowledge engineer interested in RAG.']
- **Movie**
  - `name`: STRING Available options: ['Top Gun']
  - `runtime`: INTEGER Min: 120, Max: 120
- **Actor**
  - `name`: STRING Available options: ['Tom Cruise', 'Val Kilmer', 'Anthony Edwards', 'Meg Ryan']
Relationship properties:

The relationships:
(:Person)-[:AUTHORED]->(:Document)
(:Actor)-[:ACTED_IN]->(:Movie)


In [24]:
chain = GraphCypherQAChain.from_llm(
    ChatGroq(model="llama-3.1-8b-instant", temperature=0),
    graph=graph,
    verbose=True,
    allow_dangerous_requests=True
)

In [25]:
chain.invoke({"query": "Who played in Top Gun?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (a:Actor)-[:ACTED_IN]->(m:Movie)<-[:ACTED_IN]-(b:Actor) WHERE m.name = "Top Gun" RETURN b.name
Full Context:
[{'b.name': 'Tom Cruise'}, {'b.name': 'Tom Cruise'}, {'b.name': 'Tom Cruise'}, {'b.name': 'Val Kilmer'}, {'b.name': 'Val Kilmer'}, {'b.name': 'Val Kilmer'}, {'b.name': 'Anthony Edwards'}, {'b.name': 'Anthony Edwards'}, {'b.name': 'Anthony Edwards'}, {'b.name': 'Meg Ryan'}]

> Finished chain.


{'query': 'Who played in Top Gun?',
 'result': 'Tom Cruise, Val Kilmer, Anthony Edwards, Meg Ryan played in Top Gun.'}

In [26]:
chain = GraphCypherQAChain.from_llm(
    ChatGroq(model="llama-3.1-8b-instant", temperature=0),
    graph=graph,
    verbose=True,
    top_k=2,
    allow_dangerous_requests=True,
)

In [27]:
chain.invoke({"query": "Who played in Top Gun?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (a:Actor)-[:ACTED_IN]->(m:Movie)<-[:ACTED_IN]-(b:Actor) WHERE m.name = "Top Gun" RETURN b.name
Full Context:
[{'b.name': 'Tom Cruise'}, {'b.name': 'Tom Cruise'}]

> Finished chain.


{'query': 'Who played in Top Gun?', 'result': 'Tom Cruise played in Top Gun.'}

In [ ]:
from langchain_core.prompts.prompt import PromptTemplate


CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
Examples: Here are a few examples of generated Cypher statements for particular questions:
# How many people played in Top Gun?
MATCH (m:Movie {{name:"Top Gun"}})<-[:ACTED_IN]-()
RETURN count(*) AS numberOfActors

The question is:
{question}"""

CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables = ['schema','question'], template = CYPHER_GENERATION_TEMPLATE
)


In [30]:

chain = GraphCypherQAChain.from_llm(
    ChatGroq(model = "llama-3.1-8b-instant", temperature = 0),
    graph = graph,
    verbose=True,
    cypher_prompt = CYPHER_GENERATION_PROMPT,
    allow_dangerous_requests = True,)






chain.invoke({"query": "How many people played in Top Gun?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {name:"Top Gun"})<-[:ACTED_IN]-()
RETURN count(*) AS numberOfActors
Full Context:
[{'numberOfActors': 4}]

> Finished chain.


{'query': 'How many people played in Top Gun?',
 'result': "I don't know the answer."}